In [1]:
%%configure -f
{"driverMemory": "20G",
 "executorMemory": "20G",
 "executorCores": 16,
 "numExecutors": 20,
 "name": "weighted_trend_eval_metric",
 "conf": {"spark.dynamicAllocation.enabled":"false",
          "spark.pyspark.python": "python3",
          "spark.sql.broadcastTimeout": 10000}
 }

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
164,application_1600172976670_6157,pyspark,idle,Link,Link,
167,application_1600172976670_6160,pyspark,idle,Link,Link,
168,application_1600172976670_6161,pyspark,idle,Link,Link,
172,application_1600172976670_6165,pyspark,idle,Link,Link,
173,application_1600172976670_6166,pyspark,busy,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
174,application_1600172976670_6167,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
164,application_1600172976670_6157,pyspark,idle,Link,Link,
167,application_1600172976670_6160,pyspark,idle,Link,Link,
168,application_1600172976670_6161,pyspark,idle,Link,Link,
172,application_1600172976670_6165,pyspark,idle,Link,Link,
173,application_1600172976670_6166,pyspark,busy,Link,Link,
174,application_1600172976670_6167,pyspark,idle,Link,Link,✔


In [14]:
model_date = None #keep this null if you want to use the current date
top_k = 1 
run_days = 7 #number of days to run and consider in the average, should be bigger or equal to 2

#data location
test_env = "prod" #where to read the sales i.e. l1_banner_events_base
pred_env = "prod" #where to read the model file

#model info if there are changes
model = "weighted_trendfiltering_banner_events"
category_Score_column_name = "category_trend_scores"

#uncomment the print lines to debug the issues that might occure in the calculations of this measure

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
spark.sparkContext.addPyFile("/home/hadoop/analysis/common/fileutils.py")

import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType, LongType, IntegerType, DoubleType, StructType, StructField
from datetime import datetime, timedelta, date
from fileutils import TeleScopeFileAccessUtil as fileutil

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def get_paths(model_date_out, previous_day):
    if model_date_out is None:
        model_date_in = (date.today() - timedelta(days=previous_day+1)).strftime('%Y-%m-%d')
    else:
        model_date_in = (datetime.strptime(model_date_out, '%Y-%m-%d') - timedelta(days= previous_day)).strftime('%Y-%m-%d')
    
    start_date = (datetime.strptime(model_date_in, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
    end_date = start_date
    
    print(f"model date: {model_date_in}, sales dates: {start_date}, {end_date}")

    model_predictions_path = fileutil.get_path("model", "model_path", pred_env) + f"{model}/user_categories/model_date={model_date_in}"
    ground_truth_path = fileutil.get_path("shinra", "shinra_base", test_env) + "telescope/base/l1_banner_events_base_table"
    
    return model_predictions_path, ground_truth_path, start_date, end_date, model_date_in

def model_prediction_input_checking(model_predictions, category_Score_column_name):
    if 'model_date' in model_predictions.columns:
        raise ValueError("model_date in the model prediction path, specify the model_date")

    model_predictions_schema = (model_predictions
                                .select("customer_id", category_Score_column_name)
                                .schema)

    check_schema = (StructType([StructField('customer_id', LongType(), True),
                                StructField(category_Score_column_name,
                                            ArrayType(StructType([StructField('category_id', StringType(), True),
                                                                  StructField('probability', DoubleType(), True)]),
                                                      True), True)]))

    if model_predictions_schema != check_schema:
        raise ValueError("""The prediction data customer_id should be LongType, 
            and {category_Score_column_name} should be a array which contains 
            cateogry_id(StringType) and probability(DoubleType).
            Check prediction data schema type.""".format(category_Score_column_name=category_Score_column_name))
        
def ground_truth_input_checking(ground_truth):
    ground_truth_schema = ground_truth.select("sales").schema

    check_schema = StructType([StructField('sales', LongType(), True)])

    if ground_truth_schema != check_schema:
        raise ValueError('The test data sales should be LongType. Check ground truth data schema type.')

def order_top_k(category):
    sorted_category = sorted(category, key=lambda x: x[1], reverse=True)
    top_k_category = [i[0] for i in sorted_category[:top_k]]
    return top_k_category

def get_top_user_categories(model_predictions_user_category):
    top_predictions_per_user = (model_predictions_user_category
                                .select('customer_id',
                                        F.udf(order_top_k, ArrayType(StringType()))(F.col('category')).alias(
                                            'pred_topk_cma_id')))
    return top_predictions_per_user

def get_user_sales(ground_truth, start_date, end_date):
    sales_categories = (ground_truth
                        .filter(F.col("dt").between(start_date, end_date))
                        .filter((F.col('sales').isNotNull()) & (F.col('sales') > 0))
                        .select(F.col("customer_id").cast(LongType()), "cma_id"))
    return sales_categories

def agg_sales_categories(sales_categories):
    agg_categories_per_user = (sales_categories
                               .groupBy('customer_id')
                               .agg(F.collect_set("cma_id").alias('cma_id'))
                               .select('customer_id',
                                       'cma_id',
                                       F.size(F.col("cma_id")).alias("relevant_cat_cnt")))
    return agg_categories_per_user

def count_pred_lab_k(predictions, labels, top_k):
    return len(set(labels).intersection(set(predictions[:top_k])))

def precision_at_each_user_k(relevant_documents, ks):
    precision_per_user_k = (relevant_documents
                            .crossJoin(ks)
                            .withColumn('pred_topk_cma_id', F.when(F.col('pred_topk_cma_id').isNull(), F.array([]))
                                        .otherwise(F.col('pred_topk_cma_id')))
                            .withColumn('cnt_pred_lab_k',
                                        F.udf(count_pred_lab_k)(F.col('pred_topk_cma_id'), F.col('cma_id'), F.col('k')))
                            .withColumn('perc_pred_lab_k', F.col('cnt_pred_lab_k') / F.col('k'))
                            .withColumn('recall_pred_lab_k', F.when(F.col("relevant_cat_cnt") > 0, F.col('cnt_pred_lab_k') / F.col('relevant_cat_cnt'))
                                        .otherwise(1)))
    return precision_per_user_k

def agg_precision_at_each_k(precision_per_user_k):  # fn name
    precision_per_k = (precision_per_user_k
                       .withColumn('num_label', F.size('cma_id'))
                       .withColumn('num_predictions', F.size('pred_topk_cma_id'))
                       .withColumn('num_distinct_predictions', F.size(F.array_distinct('pred_topk_cma_id')))
                       .withColumn('dup_pred_category', F.col('num_distinct_predictions') - F.col('num_predictions'))
                       .groupBy(F.col('k'))
                       .agg(F.mean('perc_pred_lab_k').alias('precision_at_k'),
                            F.mean('recall_pred_lab_k').alias("recall_at_k"),
                            F.sum('dup_pred_category').alias('dup_pred_category_at_k'),
                            F.sum('num_label').alias('num_label_at_k'))
                       .select('k', 'precision_at_k', 'recall_at_k', 'dup_pred_category_at_k', 'num_label_at_k').orderBy(F.col('k')))
    return precision_per_k

def prediction_test_error_checking(precision_per_k):
    error_checking = (precision_per_k
                      .agg(F.sum('dup_pred_category_at_k').alias('tlt_dup_pred_category'),
                           F.sum('num_label_at_k').alias('tlt_num_label'))
                      .select('tlt_dup_pred_category', 'tlt_num_label'))
    try:
        if (error_checking.collect()==None):
            raise ValueError("Empty ground truth set, check input data")
        if int(error_checking.collect()[0]['tlt_dup_pred_category']) > 0:
            raise ValueError("Duplicates in predictions, check prediction data")
        if int(error_checking.collect()[0]['tlt_num_label']) == 0:
            raise ValueError("Empty ground truth set, check input data")
    except:
        print("prediction test failed.")
        
def precision_recall_at_k_cal(relevant_documents, ks):
    # This the main function that contains every function about precision@k

    precision_per_user_k = precision_at_each_user_k(relevant_documents, ks)

    precision_per_k = agg_precision_at_each_k(precision_per_user_k).cache()

    prediction_test_error_checking(precision_per_k)

    precision_recall_at_k = precision_per_k.select('k', 'precision_at_k', 'recall_at_k')

    return precision_recall_at_k

def get_model_categories(df):
    return df.select(df.customer_id, F.explode(df.category).alias("category_scores_exp")) \
        .withColumn("cma_id", F.col("category_scores_exp").getItem("category_id")) \
        .select(F.col("cma_id").cast("long")).distinct()

def get_customers_with_sales_in_model_cats(model_categories, sales_categories):
    customers_with_sales_in_non_model_cats = (sales_categories.join(F.broadcast(model_categories
                                                                                .withColumn("isModelCat", F.lit(1))),
                                                                    on=["cma_id"], how="left")
                                              .where(F.col("isModelCat").isNull()).select("customer_id").distinct())

    customers_with_sales_in_model_cats = (sales_categories.join(F.broadcast(model_categories),
                                                                on=["cma_id"], how="inner")
                                          .select("customer_id").distinct().cache())

    num_customers_to_remove = (customers_with_sales_in_non_model_cats.join(customers_with_sales_in_model_cats
                                                                           .withColumn("isModelCat", F.lit(1)),
                                                                           on=["customer_id"], how="full")
                               .where(F.col("isModelCat").isNull()).count())

    return num_customers_to_remove, customers_with_sales_in_model_cats


def get_ranking_metrics_at_k(top_predictions_per_user, sales_categories, \
                             model_predictions_user_category, top_k, model_date):
    agg_categories_per_user = agg_sales_categories(sales_categories)

#     print("number of customers in prediction dataset: {count}".format(count=top_predictions_per_user.count()))
#     print("number of test customers in total: {count}".format(count=agg_categories_per_user.count()))

    # filter for customers that we could predict on
    model_categories = get_model_categories(model_predictions_user_category).cache()
    num_customers_to_remove, relevant_customers = get_customers_with_sales_in_model_cats(model_categories,
                                                                                         sales_categories)

#     print(
#         f"number of test customers that are removed due to not having sales in model categories: {num_customers_to_remove}")

    agg_categories_per_user = agg_categories_per_user.join(relevant_customers, on=["customer_id"], how="inner")

    relevant_documents = (agg_categories_per_user
                          .join(top_predictions_per_user, on='customer_id', how='left')
                          .select('customer_id', 'pred_topk_cma_id', 'cma_id', 'relevant_cat_cnt'))

#     print("relevant documents: {count}".format(count=relevant_documents.count()))
#     print('number of test customers are been predicted: {count}'.format(count=relevant_documents
#                                                                         .filter(F.col('cma_id').isNotNull()).count()))
#     print('number of test customers are not been predicted: {count}'.format(count=relevant_documents
#                                                                             .filter(F.col('cma_id').isNull()).count()))

    ks = spark.createDataFrame(list(range(1, top_k + 1)), IntegerType()).toDF("k")

    ks = ks.withColumn("dt", F.lit(model_date))

    precision_recall_at_k = precision_recall_at_k_cal(relevant_documents, ks)
    return precision_recall_at_k

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
def main(model_predictions_path, ground_truth_path, start_date, end_date, model_date):
    model_predictions = (spark.read.parquet(model_predictions_path))
    ground_truth = (spark.read.parquet(ground_truth_path))

    model_prediction_input_checking(model_predictions, category_Score_column_name)
    ground_truth_input_checking(ground_truth)

    model_predictions_user_category = (model_predictions
                                       .select("customer_id", F.col(category_Score_column_name).alias("category")))

    top_predictions_per_user = get_top_user_categories(model_predictions_user_category)

    sales_categories = get_user_sales(ground_truth, start_date, end_date)
    
    #print("count of categories in the model: {count}".format(count=get_model_categories(model_predictions_user_category).count()))
    
    precision_at_k = get_ranking_metrics_at_k(top_predictions_per_user, sales_categories, model_predictions_user_category,
                                              top_k, model_date).withColumn("dt", F.lit(model_date)).cache()
    
    precision_at_k.show(200, False)
    
    return precision_at_k

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
i = 1
model_predictions_path, ground_truth_path, start_date, end_date, model_date_update = get_paths(model_date, i)
precision_at_k = main(model_predictions_path, ground_truth_path, start_date, end_date, model_date_update)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model date: 2021-02-07, sales dates: 2021-02-08, 2021-02-08
+---+------------------+-------------------+----------+
|k  |precision_at_k    |recall_at_k        |dt        |
+---+------------------+-------------------+----------+
|1  |0.3860008220080404|0.37076421954573063|2021-02-07|
+---+------------------+-------------------+----------+

In [19]:
for i in range(2,run_days+1):
    print(f"rolling day num {i}: ")
    model_predictions_path, ground_truth_path, start_date, end_date, model_date_update = get_paths(model_date, i)
    precision_at_k = precision_at_k.union(main(model_predictions_path, ground_truth_path, start_date, end_date, model_date_update))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rolling day num 2: 
model date: 2021-02-06, sales dates: 2021-02-07, 2021-02-07
+---+------------------+------------------+----------+
|k  |precision_at_k    |recall_at_k       |dt        |
+---+------------------+------------------+----------+
|1  |0.5190324220081259|0.3135256228803216|2021-02-06|
+---+------------------+------------------+----------+

rolling day num 3: 
model date: 2021-02-05, sales dates: 2021-02-06, 2021-02-06
+---+------------------+------------------+----------+
|k  |precision_at_k    |recall_at_k       |dt        |
+---+------------------+------------------+----------+
|1  |0.5192057449699166|0.3176307077425393|2021-02-05|
+---+------------------+------------------+----------+

rolling day num 4: 
model date: 2021-02-04, sales dates: 2021-02-05, 2021-02-05
+---+----------------+------------------+----------+
|k  |precision_at_k  |recall_at_k       |dt        |
+---+----------------+------------------+----------+
|1  |0.52313078240778|0.3194332033970082|2021-02-

In [20]:
precision_at_k.show(200, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+-------------------+----------+
|k  |precision_at_k    |recall_at_k        |dt        |
+---+------------------+-------------------+----------+
|1  |0.3860008220080404|0.37076421954573063|2021-02-07|
|1  |0.5190324220081259|0.3135256228803216 |2021-02-06|
|1  |0.5192057449699166|0.3176307077425393 |2021-02-05|
|1  |0.52313078240778  |0.3194332033970082 |2021-02-04|
|1  |0.5624042437363542|0.34727310592133126|2021-02-03|
|1  |0.5710495843331385|0.3682475971982918 |2021-02-02|
|1  |0.5937011279862764|0.412070734233687  |2021-02-01|
+---+------------------+-------------------+----------+

In [21]:
print("overall Averages:")
precision_at_k.groupBy("k").agg(F.avg("precision_at_k").alias("precision_at_k"), 
                                F.avg("recall_at_k").alias("recall_at_k")).show(200, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

overall Averages:
+---+-----------------+-------------------+
|k  |precision_at_k   |recall_at_k        |
+---+-----------------+-------------------+
|1  |0.524932103921376|0.34984931298841576|
+---+-----------------+-------------------+